# Loading the Dataset

In [1]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
from dataset.dataset_loader import dataset_loader

train, test = dataset_loader((224, 224), is_grayscale=False)

Is the dataset balanced?

In [25]:
import numpy as np

train_chihuahuas = np.sum(train.targets)
test_chihuahuas = np.sum(test.targets)

In [28]:
print(f"We have a total of {train_chihuahuas} and {len(train) - train_chihuahuas} muffins in the training set. "
      f"This results for a ratio of {train_chihuahuas / len(train) * 100:.2f}% of chihuahuas .")
print(f"We have a total of {test_chihuahuas} and {len(test) - test_chihuahuas} muffins in the testing set."
      f"This results for a ratio of {test_chihuahuas / len(test) * 100:.2f}% of chihuahuas .")

We have a total of 2174 and 2559 muffins in the training set. This results for a ratio of 45.93% of chihuahuas .
We have a total of 544 and 640 muffins in the testing set.This results for a ratio of 45.95% of chihuahuas .


# Does KFold Split evenly?
Does my implementation of the K-Fold Cross Validation split the dataset in a desirable way?
(Meaning it is correct)

In [40]:
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper as KFoldController

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

k_fold_train = np.sum([local_train[i][1] for i in range(len(local_train))])
k_fold_test = np.sum([validation[i][1] for i in range(len(validation))])

In [42]:
print(f"We have a total of {k_fold_train} and {len(local_train) - k_fold_train} muffins in the training set. "
      f"This results for a ratio of {k_fold_train / len(local_train) * 100:.2f}% of chihuahuas .")
print(f"We have a total of {k_fold_test} and {len(validation) - k_fold_test} muffins in the testing set."
      f"This results for a ratio of {k_fold_test / len(validation) * 100:.2f}% of chihuahuas .")

We have a total of 1742 and 2044 muffins in the training set. This results for a ratio of 46.01% of chihuahuas .
We have a total of 432 and 515 muffins in the testing set.This results for a ratio of 45.62% of chihuahuas .


## Augmentation?

In [ ]:
#TBDtest_chihuahuas